<br>

# Introdução

In [1]:
#!pip3 install traquitanas --upgrade
#!pip3 install -r requirements.txt
#!/usr/local/bin/python -m pip install --upgrade pip

In [2]:
from paths import *
print(project_path)

/workspaces/acp-399-01
/workspaces/acp-399-01


In [3]:
import os
import shutil
import folium
import fnmatch
import pandas as pd
import geopandas as gpd

#from osgeo import ogr
from folium import plugins
from zipfile import ZipFile

In [4]:
from traquitanas.geo import layers as tt

<br>

Inicialmente extraímos todos os arquivos de dentro do zip "Shape.zip".

In [7]:
zipfile_path = input_path / 'Shapes.zip'

with ZipFile(zipfile_path, 'r') as zip_obj:    
    zip_obj.extractall(temp_path)

<br>

Uma vez com os arquivos soltos, foi possível observar grande desorganização dos arquivos. Não há qualquer padrão de nomenclatura. Alguns *shapefiles* existem para algumas nascentes, enquanto para outra snão existe.<br>
Como o objetivo principal do trabalho é identificar a localização das 56 nascentes que são mencionadas no TAC, optou-se por concentrar os esforços nestas feições.

In [8]:
list_shps = []

for root, dirs, files in os.walk(temp_path):
    if os.path.basename(root).startswith('Nascente'):
        print('')
        print(root)
        for file in fnmatch.filter(files, '*.shp'):
            print(f'> {file}')
            list_shps.append(file)


/workspaces/acp-399-01/data/input/temp/Nascente 37
> Area_drenagem.shp
> APP_nascente.shp
> Uso_solo_app_nasc.shp
> Geologia.shp
> Nascente.shp
> Uso_solo_area_drenagem.shp

/workspaces/acp-399-01/data/input/temp/Nascente 35
> Area_drenagem.shp
> APP_nascente.shp
> Uso_solo_app_nasc.shp
> Geologia.shp
> Nascente.shp
> Uso_solo_area_drenagem.shp

/workspaces/acp-399-01/data/input/temp/Nascente 41
> Area_drenagem.shp
> APP_nascente.shp
> Uso_solo_app_nasc.shp
> Geologia.shp
> Nascente.shp
> Uso_solo_area_drenagem.shp

/workspaces/acp-399-01/data/input/temp/Nascente 49
> Area_drenagem.shp
> APP_nascente.shp
> Uso_solo_app_nasc.shp
> Geologia.shp
> Nascente.shp
> Uso_solo_area_drenagem.shp

/workspaces/acp-399-01/data/input/temp/Nascente 14
> Area_drenagem.shp
> APP.shp
> Uso_solo_app_nasc.shp
> Area_conservacao.shp
> Geologia.shp
> Nascente.shp
> Uso_solo_APP.shp
> Uso_solo_area_drenagem.shp

/workspaces/acp-399-01/data/input/temp/Nascente 30
> Area_drenagem.shp
> APP_nascente.shp
> Uso_

In [9]:
list_shps = list(set(list_shps))
list_shps.sort()
list_shps

['APP.shp',
 'APP_nascente.shp',
 'APP_total.shp',
 'Area_Drenagem.shp',
 'Area_conservacao.shp',
 'Area_drenagem.shp',
 'Geologia.shp',
 'Nascente.shp',
 'Pontos_geofisica.shp',
 'Recomposicao.shp',
 'Recuperacao_curso_agua.shp',
 'Recuperacao_curso_dagua.shp',
 'Recuperacao_cursodagua.shp',
 'Recuperacao_nascente.shp',
 'Uso_solo_APP.shp',
 'Uso_solo_app_curso_dagua.shp',
 'Uso_solo_app_nasc.shp',
 'Uso_solo_area_drenagem.shp',
 'Uso_solo_curso_agua.shp',
 'Uso_solo_hidrografia.shp',
 'area_Conservacao.shp',
 'area_drenagem.shp']

<br>

## Junta shapefiles por Nome

Com a lista dos possíveis nomes de shapefiles, juntou-se todos os arquivos.

In [10]:
def merge_shp_by_name(shp, path):
    """
    path: pasta root que tem os shapefiles
    
    """
    # Add Fields
    print('\n{}'.format(shp))
    for root, dirs, files in os.walk(path):
        if os.path.basename(root).startswith('Nascente'):
            for file in fnmatch.filter(files, '*.shp'):
                if file in [shp]:
                    # Read
                    temp = os.path.join(root, file)
                    gdf = gpd.read_file(temp)
                    gdf.columns = [x.lower() for x in gdf.columns]

                    # Delete Fields
                    #for col in ['Id', 'path', 'nascente']:
                    gdf.drop([
                        'Id', 'path',
                        'nascente', 'Nascente',
                        'numero', 'x', 'y',
                        'BUFF_DIST', 'buff_dist',
                        'nascente_1'
                        'nascente_1', 'nascente_2',
                        'nascente_3', 'nascente_4',
                        'nascente_5'],
                        axis=1,
                        errors='ignore',
                        inplace=True,
                    )
                    
                    # Add Fields
                    gdf['path'] = temp
                    dirname = os.path.dirname(temp)
                    dirname = os.path.basename(dirname)
                    nascente_number = int(dirname.split(' ', maxsplit=1)[-1])
                    gdf['nascente'] = nascente_number

                    # Save Output
                    gdf.to_file(temp)

    # Lista Files
    print(gdf.columns)
    list_files = []                
    for root, dirs, files in os.walk(path):
        if os.path.basename(root).startswith('Nascente'):
            for file in fnmatch.filter(files, '*.shp'):
                if file in [shp]:
                    temp = os.path.join(root, file)
                    list_files.append(temp)

    # Combine all shapefiles
    gdf = gpd.GeoDataFrame(
        pd.concat(
            [gpd.read_file(i) for i in list_files],
            ignore_index=True
        ),
        crs=gpd.read_file(list_files[0]).crs
    )

    # Save Output
    gdf.to_file(os.path.join(output_path, shp))
    display(gdf)
    return gdf

In [11]:
for shp in list_shps:
    gdf = merge_shp_by_name(shp, temp_path)


APP.shp
Index(['id', 'prioridade', 'area', 'geometry', 'path', 'nascente'], dtype='object')


,id,prioridade,area,path,nascente,geometry
0,0,1,7833.33,/workspaces/acp-399-01/data/input/temp/Nascent...,14,"POLYGON ((248255.632 7515114.713, 248261.899 7..."
1,0,1,7833.33,/workspaces/acp-399-01/data/input/temp/Nascent...,5,"POLYGON ((248801.925 7517497.654, 248808.192 7..."
2,0,1,7833.33,/workspaces/acp-399-01/data/input/temp/Nascent...,7,"POLYGON ((249280.337 7516753.212, 249286.604 7..."
3,0,1,7833.33,/workspaces/acp-399-01/data/input/temp/Nascent...,12,"POLYGON ((251858.104 7514488.289, 251864.371 7..."
4,0,1,7833.33,/workspaces/acp-399-01/data/input/temp/Nascent...,2,"POLYGON ((248131.028 7516717.435, 248137.295 7..."
5,0,1,7833.33,/workspaces/acp-399-01/data/input/temp/Nascent...,10,"POLYGON ((250902.507 7516125.816, 250908.773 7..."
6,0,1,7833.33,/workspaces/acp-399-01/data/input/temp/Nascent...,6,"POLYGON ((249086.609 7517066.833, 249092.876 7..."
7,0,1,7833.33,/workspaces/acp-399-01/data/input/temp/Nascent...,3,"POLYGON ((248148.184 7516816.967, 248154.450 7..."
8,0,1,7833.33,/workspaces/acp-399-01/data/input/temp/Nascent...,11,"POLYGON ((251190.751 7515714.262, 251197.018 7..."
9,0,1,7833.33,/workspaces/acp-399-01/data/input/temp/Nascent...,8,"POLYGON ((249099.808 7516533.489, 249106.075 7..."



APP_nascente.shp
Index(['id', 'prioridade', 'geometry', 'path', 'nascente'], dtype='object')


,id,prioridade,path,nascente,geometry
0,0,2,/workspaces/acp-399-01/data/input/temp/Nascent...,37,"POLYGON ((245346.435 7512209.416, 245352.701 7..."
1,0,2,/workspaces/acp-399-01/data/input/temp/Nascent...,35,"POLYGON ((245301.958 7511848.120, 245308.224 7..."
2,0,2,/workspaces/acp-399-01/data/input/temp/Nascent...,41,"POLYGON ((250025.970 7510431.678, 250032.237 7..."
3,0,2,/workspaces/acp-399-01/data/input/temp/Nascent...,49,"POLYGON ((249210.992 7510649.188, 249217.259 7..."
4,0,2,/workspaces/acp-399-01/data/input/temp/Nascent...,30,"POLYGON ((244920.865 7508590.985, 244927.132 7..."
5,0,2,/workspaces/acp-399-01/data/input/temp/Nascent...,46,"POLYGON ((251969.603 7508302.439, 251975.870 7..."
6,0,2,/workspaces/acp-399-01/data/input/temp/Nascent...,38,"POLYGON ((249291.152 7512206.506, 249297.418 7..."
7,0,2,/workspaces/acp-399-01/data/input/temp/Nascent...,19,"POLYGON ((252556.721 7512267.881, 252562.988 7..."
8,0,2,/workspaces/acp-399-01/data/input/temp/Nascent...,43,"POLYGON ((251505.927 7510071.764, 251512.193 7..."
9,0,2,/workspaces/acp-399-01/data/input/temp/Nascent...,24,"POLYGON ((244859.110 7507199.748, 244865.377 7..."



APP_total.shp
Index(['id', 'nome', 'geometry', 'path', 'nascente'], dtype='object')


...

,id,area,uso,descricao,path,nascente,geometry
0,0,29144.00,7,Vegetação em regeneração,/workspaces/acp-399-01/data/input/temp/30_03_1...,9,"MULTIPOLYGON (((250237.985 7515543.349, 250230..."
1,0,37182.40,1,Vegetação Nativa,/workspaces/acp-399-01/data/input/temp/30_03_1...,9,"MULTIPOLYGON (((250245.869 7515649.024, 250237..."
2,0,58092.80,2,Cana,/workspaces/acp-399-01/data/input/temp/30_03_1...,9,"MULTIPOLYGON (((250188.031 7515601.436, 250181..."
3,0,945.69,6,Agricultura,/workspaces/acp-399-01/data/input/temp/30_03_1...,9,"POLYGON ((250284.375 7515954.762, 250250.488 7..."
4,0,5549.55,5,Pastagem,/workspaces/acp-399-01/data/input/temp/30_03_1...,9,"POLYGON ((250254.020 7515743.693, 250253.970 7..."



area_Conservacao.shp
Index(['id', 'area', 'geometry', 'path', 'nascente'], dtype='object')


,id,area,path,nascente,geometry
0,0,589558.0,/workspaces/acp-399-01/data/input/temp/Nascent...,4,"POLYGON ((248224.705 7517398.803, 248224.705 7..."



area_drenagem.shp
Index(['id', 'area', 'geometry', 'path', 'nascente'], dtype='object')


,id,area,path,nascente,geometry
0,0,593202.0,/workspaces/acp-399-01/data/input/temp/Nascent...,4,"POLYGON ((248273.819 7517389.434, 248176.970 7..."


<br>

## Junta shapefiles Específicos

Após juntar todos os shapefiles em pastas, notei a existência de shapefiles que tem o mesmo tema, porém continham nomes distintos (por exemplo, "app.shp" e "App.shp").<br>
Esse problema na nomenclatura dos arquivos, impediu que os arquivos fossem unificados e, portanto, foi necessário escrever uma função que juntava os shapefiles, mantendo apenas o primeiro e deletando os restantes.

In [ ]:
def merge_shp_specific(list_files):
    """
    Une shapefiles específicos, sobrepondo o primeiro da lista...
    Deletando os demais...
    """
    # Concatena
    gdf = gpd.GeoDataFrame(
        pd.concat(
            [gpd.read_file(i) for i in list_files],
            ignore_index=True
        ),
        crs=gpd.read_file(list_files[0]).crs
    )

    # Deleta Others Files
    driver = ogr.GetDriverByName('ESRI Shapefile')
    for file in list_files[1:]:
        if os.path.exists(file):
            driver.DeleteDataSource(file)
            print('"{file}" deletado!!')
        else:
            print(f'"{file}" não existe!')

    # Save Output
    gdf.to_file(os.path.join(list_files[0]))

In [ ]:
merge_shp_specific(
    [
        os.path.join(output_path, 'APP.shp'),
        os.path.join(output_path, 'APP_nascente.shp'),
    ]
)

In [ ]:
merge_shp_specific(
    [
        os.path.join(output_path, 'Area_conservacao.shp'),
        os.path.join(output_path, 'area_Conservacao.shp'),
    ]
)

In [ ]:
merge_shp_specific(
    [
        os.path.join(output_path, 'area_drenagem.shp'),
        os.path.join(output_path, 'Area_Drenagem.shp'),
        os.path.join(output_path, 'Area_drenagem.shp'),        
    ]
)

In [ ]:
merge_shp_specific(
    [
        os.path.join(output_path, 'Recuperacao_curso_agua.shp'),
        os.path.join(output_path, 'Recuperacao_curso_dagua.shp'),
        os.path.join(output_path, 'Recuperacao_cursodagua.shp'),        
    ]
)

<br>

## Deleta Shapefiles

Após juntar o material que considerei útil para a etapa do meu trabalho, foi possível deletar o restante do material.

In [ ]:
def delete_shp(list_files):
    """
    Delete Lista de Shapefiles
    """
    # Deleta Others Files
    driver = ogr.GetDriverByName('ESRI Shapefile')
    for file in list_files[1:]:
        if os.path.exists(file):
            driver.DeleteDataSource(file)
            print(f'"{file}" deletado!!')
        else:
            print(f'"{file}" não existe!')


In [ ]:
list_files = [
    # Lixo
    os.path.join(output_path, 'APP_total.shp'),

    # Interessa?
    os.path.join(output_path, 'Geologia.shp'),
    #os.path.join(output_path, 'Nascente.shp'),
    os.path.join(output_path, 'Pontos_geofisica.shp'),
    os.path.join(output_path, 'Recomposicao.shp'),
    os.path.join(output_path, 'Recuperacao_nascente.shp'),
    os.path.join(output_path, 'Uso_solo_APP.shp'),
    os.path.join(output_path, 'Uso_solo_app_curso_dagua.shp'),
    os.path.join(output_path, 'Uso_solo_app_nasc.shp'),
    os.path.join(output_path, 'Uso_solo_area_drenagem.shp'),
    os.path.join(output_path, 'Uso_solo_curso_agua.shp'),
    os.path.join(output_path, 'Uso_solo_hidrografia.shp'),
]

delete_shp(list_files)

In [ ]:
shutil.rmtree(temp_path, ignore_errors=True)

<br>

# Maps

<br>

## Layer: Nascentes

In [ ]:
def add_lyr_nascentes(output_path):
    # Input
    gdf = gpd.read_file(os.path.join(output_path, 'Nascente.shp'))
    gdf = gdf.to_crs(epsg=4326)
    
    # Layers
    lyr = folium.GeoJson(
        gdf,
        name='Nascentes',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': '#DC143C',
            'color': '#DC143C',
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['nascente'],
            aliases=['Nascente'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
    #     popup=folium.GeoJsonPopup(
    #         ['popup'],
    #         parse_html=False,
    #         max_width='400',
    #         show=False,
    #         labels=False,
    #         sticky=True,            
    #     ),
        embed=False,
        zoom_on_click=False,
        control=True,
        show=True,
    )
    return lyr

<br>

## Layer: Área Contribuição

In [ ]:
def add_lyr_areadrenagem(output_path):
    # Input
    gdf = gpd.read_file(os.path.join(output_path, 'area_drenagem.shp'))
    gdf = gdf.to_crs(epsg=4326)
    
    # Layers
    lyr = folium.GeoJson(
        gdf,
        name='Área de Contribuição',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': '#DC143C',
            'color': '#DC143C',
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['nascente'],
            aliases=['Nascente'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
    #     popup=folium.GeoJsonPopup(
    #         ['popup'],
    #         parse_html=False,
    #         max_width='400',
    #         show=False,
    #         labels=False,
    #         sticky=True,            
    #     ),
        embed=False,
        zoom_on_click=False,
        control=True,
        show=True,
    )
    return lyr

<br>

## Maps

In [ ]:
def get_map(input_shp):
    # Input
    gdf = gpd.read_file(input_shp)
    gdf = gdf.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]
    
    # Zoom
    min_zoom = 10
    max_zoom = 18
    
    # Create Map
    m = folium.Map(
        #zoom_start=10,
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        max_bounds=True,
        #zoom_delta=0.1,s
        min_lat=bounds[0][0]*(101/100),
        min_lon=bounds[0][1]*(101/100),
        max_lat=bounds[1][0]*(99/100),
        max_lon=bounds[1][1]*(99/100),
        tiles=None,
    )
    
    # Add Layers
    m.add_child(tt.add_lyr_google_terrain(min_zoom, max_zoom))
    m.add_child(tt.add_lyr_google_hybrid(min_zoom, max_zoom))
    m.add_child(tt.add_lyr_google_streets(min_zoom, max_zoom))
    m.add_child(tt.add_lyr_google_satellite(min_zoom, max_zoom))

    # Outros Layers
    m.add_child(add_lyr_nascentes(output_path))
    m.add_child(add_lyr_areadrenagem(output_path))
    #m.add_child(add_lyr_car())
    #m.add_child(add_lyr_divisa_municipal())
    #m.add_child(add_lyr_macrozona())
    #m.add_child(add_lyr_wms())
    
    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False
    ).add_to(m)
    return m

In [ ]:
# Create Map
m = get_map(os.path.join(shps_path, 'Nascente.shp'))

# Results
m.save(os.path.join(maps_path, 'acp-399-01_map.html'))
m